# Chapitre 7 : Créer des applications de chat
## Démarrage rapide avec l’API Github Models

Ce notebook est adapté du [dépôt d’exemples Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) qui comprend des notebooks permettant d’accéder aux services [Azure OpenAI](notebook-azure-openai.ipynb).


# Vue d'ensemble  
« Les grands modèles de langage sont des fonctions qui associent un texte à un autre texte. Lorsqu’on leur fournit une chaîne de texte en entrée, un grand modèle de langage tente de prédire le texte qui suivra »(1). Ce carnet « démarrage rapide » présente aux utilisateurs les concepts clés des LLM, les principaux prérequis pour commencer avec AML, une introduction simplifiée à la conception de prompts, ainsi que plusieurs exemples courts illustrant différents cas d’usage.


## Table des matières  

[Aperçu](../../../../07-building-chat-applications/python)  
[Comment utiliser le service OpenAI](../../../../07-building-chat-applications/python)  
[1. Créer votre service OpenAI](../../../../07-building-chat-applications/python)  
[2. Installation](../../../../07-building-chat-applications/python)    
[3. Identifiants](../../../../07-building-chat-applications/python)  

[Cas d’utilisation](../../../../07-building-chat-applications/python)    
[1. Résumer un texte](../../../../07-building-chat-applications/python)  
[2. Classer un texte](../../../../07-building-chat-applications/python)  
[3. Générer de nouveaux noms de produits](../../../../07-building-chat-applications/python)  
[4. Affiner un classificateur](../../../../07-building-chat-applications/python)  

[Références](../../../../07-building-chat-applications/python)


### Créez votre première invite  
Ce court exercice vous donnera une introduction de base à la soumission d'invites à un modèle dans Github Models pour une tâche simple de "résumé".


**Étapes** :  
1. Installez la bibliothèque `azure-ai-inference` dans votre environnement python, si ce n'est pas déjà fait.  
2. Chargez les bibliothèques d'aide standard et configurez les identifiants pour Github Models.  
3. Choisissez un modèle pour votre tâche  
4. Rédigez une invite simple pour le modèle  
5. Envoyez votre requête à l'API du modèle !


### 1. Installez `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Trouver le bon modèle  
Les modèles GPT-3.5-turbo ou GPT-4 peuvent comprendre et générer du langage naturel.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Conception de prompts  

« La magie des grands modèles de langage, c’est qu’en étant entraînés à minimiser cette erreur de prédiction sur d’immenses quantités de texte, les modèles finissent par apprendre des concepts utiles pour ces prédictions. Par exemple, ils apprennent des notions comme »(1) :

* comment épeler
* comment fonctionne la grammaire
* comment reformuler
* comment répondre à des questions
* comment tenir une conversation
* comment écrire dans de nombreuses langues
* comment coder
* etc.

#### Comment contrôler un grand modèle de langage  
« Parmi toutes les entrées d’un grand modèle de langage, c’est de loin le prompt textuel qui a le plus d’influence »(1).

On peut inciter les grands modèles de langage à produire des résultats de plusieurs façons :

Instruction : Dites au modèle ce que vous voulez  
Complétion : Amenez le modèle à compléter le début de ce que vous souhaitez  
Démonstration : Montrez au modèle ce que vous attendez, soit avec :  
Quelques exemples dans le prompt  
Des centaines ou des milliers d’exemples dans un jeu de données d’entraînement pour l’ajustement fin »



#### Il existe trois règles de base pour créer des prompts :

**Montrer et expliquer**. Soyez clair sur ce que vous attendez, que ce soit par des instructions, des exemples, ou une combinaison des deux. Si vous voulez que le modèle classe une liste d’éléments par ordre alphabétique ou qu’il catégorise un paragraphe selon le sentiment, montrez-lui ce que vous attendez.

**Fournir des données de qualité**. Si vous essayez de créer un classificateur ou d’amener le modèle à suivre un schéma, assurez-vous d’avoir suffisamment d’exemples. Relisez bien vos exemples — le modèle est généralement assez intelligent pour repérer les fautes d’orthographe et vous donner une réponse, mais il pourrait aussi supposer que c’est intentionnel, ce qui peut influencer la réponse.

**Vérifiez vos paramètres.** Les paramètres temperature et top_p contrôlent à quel point le modèle est déterministe dans la génération de la réponse. Si vous attendez une réponse unique et correcte, il vaut mieux les régler plus bas. Si vous souhaitez des réponses plus variées, vous pouvez les augmenter. L’erreur la plus fréquente avec ces paramètres est de croire qu’ils contrôlent « l’intelligence » ou la « créativité » du modèle.


Source : https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Résumer un texte  
#### Défi  
Résumez un texte en ajoutant un 'tl;dr:' à la fin d’un passage. Remarquez comment le modèle comprend comment accomplir plusieurs tâches sans instructions supplémentaires. Vous pouvez essayer des consignes plus détaillées que tl;dr pour modifier le comportement du modèle et personnaliser le résumé que vous obtenez(3).  

Des travaux récents ont montré des progrès importants sur de nombreuses tâches et benchmarks en NLP grâce à une pré-formation sur un vaste corpus de textes, suivie d’un ajustement sur une tâche spécifique. Bien que cette méthode soit généralement indépendante de la tâche dans son architecture, elle nécessite tout de même des jeux de données d’ajustement spécifiques à la tâche, comportant des milliers ou des dizaines de milliers d’exemples. À l’inverse, les humains peuvent généralement accomplir une nouvelle tâche linguistique avec seulement quelques exemples ou de simples instructions – ce que les systèmes NLP actuels ont encore du mal à faire. Ici, nous montrons qu’augmenter la taille des modèles de langage améliore nettement les performances indépendantes de la tâche, même avec peu d’exemples, et peut parfois rivaliser avec les meilleures approches d’ajustement précédentes. 



Tl;dr


# Exercices pour plusieurs cas d’utilisation  
1. Résumer un texte  
2. Classer un texte  
3. Générer de nouveaux noms de produits


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Classer un texte  
#### Défi  
Classer des éléments dans des catégories fournies au moment de l’inférence. Dans l’exemple suivant, nous donnons à la fois les catégories et le texte à classer dans l’invite (*playground_reference).

Demande du client : Bonjour, une des touches de mon clavier d’ordinateur portable s’est cassée récemment et j’aurais besoin d’un remplacement :

Catégorie classée :


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Générer de nouveaux noms de produits
#### Défi
Créez des noms de produits à partir de mots exemples. Ici, nous incluons dans l’invite des informations sur le produit pour lequel nous allons générer des noms. Nous fournissons également un exemple similaire pour montrer le modèle que nous souhaitons obtenir. Nous avons aussi réglé la valeur de la température à un niveau élevé pour augmenter l’aléatoire et obtenir des réponses plus innovantes.

Description du produit : Un appareil à milkshake pour la maison
Mots de base : rapide, sain, compact.
Noms de produits : HomeShaker, Fit Shaker, QuickShake, Shake Maker

Description du produit : Une paire de chaussures qui s’adapte à toutes les tailles de pied.
Mots de base : adaptable, ajustement, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Références  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Exemples OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Bonnes pratiques pour l’ajustement fin de GPT-3 pour la classification de texte](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Pour plus d’aide  
[Équipe de commercialisation OpenAI](AzureOpenAITeam@microsoft.com)


# Contributeurs
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction par IA [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour les informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
